<a href="https://colab.research.google.com/github/Abdulrahman-Aljasser/My-projects/blob/main/%D9%85%D8%B1%D8%B4%D8%AF_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv
!pip install python-telegram-bot pandas
import nest_asyncio
nest_asyncio.apply()
app.run_polling(non_blocking=True)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 8.8 MB/s eta 0:00:00


NameError: name 'app' is not defined

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
import os
from telegram import Update, ReplyKeyboardMarkup, ReplyKeyboardRemove
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters
from dotenv import load_dotenv
import asyncio

# تحميل البيئة
load_dotenv()
TOKEN = "7766299556:AAF0hjxA63YSBht15hVLq6OrasM8Rowgzxk"  # ضع التوكن الخاص بك هنا

BASE_DIR = '/content'  # تعيين مسار المجلد في Google Colab

# تحميل البيانات
data_courses = pd.read_excel(os.path.join(BASE_DIR, 'المواد.xlsx'))
data_sections = pd.read_excel(os.path.join(BASE_DIR, 'الشعب.xlsx'))

# تنظيف البيانات
data_courses.columns = data_courses.columns.str.strip()
data_sections.columns = data_sections.columns.str.strip()

# قائمة المواد الاختيارية لكل تخصص
optional_files = {
    'MIS': 'المواد الاختياريةmis.xlsx',
    'MGT': 'المواد الاختياريةmgt.xlsx',
    'MKT': 'المواد الاختياريةmkt.xlsx',
    'FIN': 'المواد الاختياريةfin.xlsx',
    'ACC': 'المواد الاختياريةacc.xlsx'
}

optional_data = {}
for major, filename in optional_files.items():
    file_path = os.path.join(BASE_DIR, filename)
    if os.path.exists(file_path):
        try:
            df = pd.read_excel(file_path)
            df.columns = df.columns.str.strip()
            optional_data[major] = df
        except Exception as e:
            print(f"حدث خطأ أثناء تحميل المواد الاختيارية لـ {major}: {e}")
    else:
        optional_data[major] = None

# التعامل مع بداية البوت
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    keyboard = [['📚 استعلام عن مادة', '📋 استعلام عن شعبة'],
                ['📖 الخطة الدراسية', '📖 دليل خدمات الطالب'],
                ['📋 المواد الاختيارية', '❌ خروج']]
    reply_markup = ReplyKeyboardMarkup(keyboard, resize_keyboard=True)
    await update.message.reply_text(
        "✉️ مرحبًا بك في بوت خدمة الطالب!\nاختر الخدمة من الأزرار أو اكتب /start للقائمة الرئيسية.\n- استعلام عن المواد والشُعب.\n- استكشاف الخطة الدراسية.\n- دليل خدمات الطالب.\n- استكشاف المواد الاختيارية.\n🚀 جاهزين نخدمك دائمًا!",
        reply_markup=reply_markup
    )

# التعامل مع قائمة المواد الاختيارية
async def handle_optional_menu(update: Update, context: ContextTypes.DEFAULT_TYPE):
    keyboard = [['تخصص MIS', 'تخصص MGT'],
                ['تخصص MKT', 'تخصص FIN'],
                ['تخصص ACC', '🔙 رجوع']]
    reply_markup = ReplyKeyboardMarkup(keyboard, resize_keyboard=True)
    await update.message.reply_text("📋 اختر القسم الذي تريد عرض مواده الاختيارية:", reply_markup=reply_markup)

# عرض المواد الاختيارية حسب التخصص
async def show_optional_by_major(update: Update, major: str):
    df = optional_data.get(major)
    if df is None:
        await update.message.reply_text("🚧 المواد الاختيارية لهذا التخصص لم تُضاف بعد.")
        return

    msg = f"📋 *المواد الاختيارية لتخصص {major}:*\n"
    for _, row in df.iterrows():
        msg += (
            f"- {row['رمز المادة']} - {row['اسم المادة']}\n"
            f"  • المتطلبات: {row['المتطلبات']}\n"
            f"  • عدد الساعات: {row['عدد الساعات']}\n\n"
        )
    await update.message.reply_text(msg, parse_mode='Markdown')

# التعامل مع مدخلات لوحة المفاتيح
async def handle_keyboard_input(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text.strip().upper()  # تحويل النص إلى حروف كبيرة وإزالة المسافات
    awaiting = context.user_data.get('awaiting')

    # استعلام عن شعبة
    if text == '📋 استعلام عن شعبة':
        await update.message.reply_text("💬 اكتب رمز المقرر للاستعلام عن جميع شعبه، مثل: MGT408")
        context.user_data['awaiting'] = 'section_info'

    elif awaiting == 'section_info':
        context.user_data['awaiting'] = None
        course_code = text.strip()  # التأكد من إزالة المسافات الزائدة
        sections = data_sections[data_sections['رمزالمقرر'].str.upper().str.strip() == course_code]  # البحث باستخدام حروف كبيرة

        if not sections.empty:
            msg = f"📋 *تفاصيل المقرر {course_code}:*\n"
            for _, sec in sections.iterrows():
                msg += (
                    f"\n🔹 رقم الشعبة: {sec['الشعبة']}\n"
                    f"🔹 الرقم المرجعي: {sec['الرقم المرجعي']}\n"
                    f"🔹 اسم المقرر: {sec['أسم المقرر']}\n"
                    f"🔹 نوع الشعبة: {sec['نوع الشعبة']}\n"
                    f"🔹 الوقت: {sec['الوقت']}\n"
                    f"🔹 الأيام: {sec['الايام']}\n"
                    f"🔹 المبنى: {sec['المبنى']}\n"
                    f"🔹 رقم القاعة: {sec['رقم القاعة']}\n\n"
                )
            keyboard = [['استعلام عن شعبة أخرى', '🔙 رجوع']]
            reply_markup = ReplyKeyboardMarkup(keyboard, resize_keyboard=True)
            await update.message.reply_text(msg, parse_mode='Markdown', reply_markup=reply_markup)
        else:
            await update.message.reply_text("⚠️ لا توجد شعب لهذا المقرر أو الرمز غير صحيح.")

    # استعلام عن شعبة أخرى
    elif text == 'استعلام عن شعبة أخرى':
        context.user_data['awaiting'] = 'section_info'
        await update.message.reply_text("💬 اكتب رمز المقرر للاستعلام عن شعبته، مثل: MGT408")

    # استعلام عن مادة
    elif text == '📚 استعلام عن مادة':
        await update.message.reply_text("💬 اكتب رمز المادة للاستعلام عن تفاصيلها، مثل: MIS101")
        context.user_data['awaiting'] = 'course_info'

    # استعلام عن مادة أخرى
    elif text == 'استعلام عن مادة أخرى':
        # إعادة تعيين حالة الانتظار إلى "استعلام عن مادة"
        context.user_data['awaiting'] = 'course_info'
        await update.message.reply_text("💬 اكتب رمز المادة للاستعلام عن تفاصيلها، مثل: MIS101")

    elif awaiting == 'course_info':
        context.user_data['awaiting'] = None
        course = data_courses[data_courses['رمز المادة'] == text]
        if not course.empty:
            row = course.iloc[0]
            msg = f"🔹 *{row['اسم المادة']}*\nمتطلبات: {row['المتطلبات']}\nعدد الساعات: {row['عدد الساعات']}"
            keyboard = [['استعلام عن مادة أخرى', '🔙 رجوع']]
            reply_markup = ReplyKeyboardMarkup(keyboard, resize_keyboard=True)
            await update.message.reply_text(msg, parse_mode='Markdown', reply_markup=reply_markup)
        else:
            await update.message.reply_text("⚠️ رمز المادة غير صحيح.")

    # المواد الاختيارية
    elif text == '📋 المواد الاختيارية':
        await handle_optional_menu(update, context)

    elif text in ['تخصص MIS', 'تخصص MGT', 'تخصص MKT', 'تخصص FIN', 'تخصص ACC']:
        major_map = {
            'تخصص MIS': 'MIS',
            'تخصص MGT': 'MGT',
            'تخصص MKT': 'MKT',
            'تخصص FIN': 'FIN',
            'تخصص ACC': 'ACC'
        }
        await show_optional_by_major(update, major_map[text])

    # الخطة الدراسية
    elif text == '📖 الخطة الدراسية':
        keyboard = [['2020 فما قبل', '2021 - 2023', '2024'], ['🔙 رجوع']]
        reply_markup = ReplyKeyboardMarkup(keyboard, resize_keyboard=True)
        context.user_data['awaiting'] = 'choose_plan_year'
        await update.message.reply_text("📖 اختر دفعتك لعرض الخطة الدراسية المناسبة:", reply_markup=reply_markup)

    # إرسال الخطة الدراسية بناءً على اختيار المستخدم
    elif awaiting == 'choose_plan_year':
        context.user_data['awaiting'] = None
        if text == '2020 فما قبل':
            filename = 'الخطة 2020 فما قبل.pdf'
        elif text == '2021 - 2023':
            filename = 'الخطة 2021 - 2023.pdf'
        elif text == '2024':
            filename = 'الخطة 2024.pdf'
        else:
            await update.message.reply_text("❌ خيار غير صحيح، حاول مرة أخرى.")
            return

        file_path = os.path.join(BASE_DIR, filename)
        if os.path.exists(file_path):
            await update.message.reply_document(
                document=open(file_path, 'rb'),
                caption=f"📖 هذه هي الخطة الدراسية لدفعة {text} ✅"
            )
        else:
            await update.message.reply_text("⚠️ ملف الخطة غير موجود بجانب ملف البوت.")

    # دليل خدمات الطالب
    elif text == '📖 دليل خدمات الطالب':
        keyboard = [['تسجيل المقررات', 'سحب مادة'], ['🔙 رجوع']]
        reply_markup = ReplyKeyboardMarkup(keyboard, resize_keyboard=True)
        context.user_data['awaiting'] = 'student_guide'
        await update.message.reply_text("📖 اختر الخدمة التي تحتاج دليلها:", reply_markup=reply_markup)

    # سحب مادة وتسجيل المقررات
    elif awaiting == 'student_guide':
        file_map = {
            'تسجيل المقررات': 'تسجيل المقررات عبر منصة الارشاد الاكاديمي.pdf',
            'سحب مادة': 'طريقة سحب مادة.pdf'
        }

        if text in file_map:
            filename = file_map[text]
            file_path = os.path.join(BASE_DIR, filename)
            if os.path.exists(file_path):
                await update.message.reply_document(
                    document=open(file_path, 'rb'),
                    caption=f"📖 هذا هو الدليل المطلوب ✅"
                )
            else:
                await update.message.reply_text("⚠️ ملف الدليل غير موجود بجانب ملف البوت.")
        elif text == '🔙 رجوع':
            await start(update, context)
        else:
            await update.message.reply_text("❌ خيار غير صحيح، حاول مرة أخرى.")

    # العودة
    elif text == '🔙 رجوع':
        await start(update, context)

    # الخروج
    elif text == '❌ خروج':
        await update.message.reply_text("🚪 تم إنهاء الجلسة، لخدمتك مرة أخرى اضغط /start", reply_markup=ReplyKeyboardRemove())




# تشغيل البوت باستخدام asyncio
if __name__ == '__main__':
    app = ApplicationBuilder().token(TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_keyboard_input))
    print("✅ البوت يعمل الآن بكامل المزايا...")
    asyncio.run(app.run_polling())


✅ البوت يعمل الآن بكامل المزايا...


ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.11/asyncio/tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 658